In [25]:
import pandas as pd
import missingno as msn
import statsmodels.api         as sm
import statsmodels.formula.api as smf
import seaborn as sns
import numpy as np

In [26]:
df = pd.read_csv('C:/Users/User/Downloads/ds4a-secopII/SECOP_Electronicos_Cleaned.csv',  parse_dates=['Fecha de Inicio del Contrato'])

In [27]:
print('The shape of the dataset result of data cleaning is : ' + str(df.shape))

The shape of the dataset result of data cleaning is : (1253547, 26)


In [28]:
df.columns.to_list()

['Departamento',
 'Orden',
 'Sector',
 'Rama',
 'Entidad Centralizada',
 'Estado Contrato',
 'Tipo de Contrato',
 'Modalidad de Contratacion',
 'Fecha de Inicio del Contrato',
 'Fecha de Fin del Contrato',
 'Es Grupo',
 'Es Pyme',
 'Obligación Ambiental',
 'Obligaciones Postconsumo',
 'Valor del Contrato',
 'Valor de pago adelantado',
 'Valor Facturado',
 'Valor Pendiente de Pago',
 'Valor Pagado',
 'Valor Amortizado',
 'Valor Pendiente de Amortizacion',
 'Valor Pendiente de Ejecucion',
 'EsPostConflicto',
 'Destino Gasto',
 'Origen de los Recursos',
 'Dias Adicionados']

In [ ]:

formula = 'Delay ~ C(Departamento) + Q("Valor del Contrato") + C(Q("Modalidad de Contratacion"))'
model_all = smf.ols(formula = formula, data = df).fit()
print(model_all.summary())